In [1]:
import re
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
#import plotly.graph_objects as go
#import plotly.express as px
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By

#from optimize_lineup import Optimized_Lineups
from fantasy_utils import Fantasy_Projections as fp
import fantasy_utils as fu
fp = fu.Fantasy_Projections()

In [2]:
def login():
    driver = webdriver.Chrome("C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe")
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    search_box = driver.find_element_by_id('userid')
    search_box.send_keys('gostros09')
    search_box = driver.find_element_by_id('password')
    search_box.send_keys('Segneri9A')
    search_box.submit()
    return driver

In [3]:
def scrape_cbs_owner_weekly_roster(soup, ids, owner, wk):
    id_dict = {i.text:int(i['href'].split('/')[-1]) for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
    df = pd.DataFrame()
    for idx in range(2):
        t = pd.read_html(str(soup.find_all('table')[0:24]))[idx]
        t.columns = t.columns.droplevel().droplevel()
        t = t[['Pos', 'Players', 'Salary']]
        t['Week'] = wk
        t['Owner'] = owner
        
        if idx==0: # for batters
            reserves_idx = t[t['Pos']=='Reserves'].index.values[0]
            t = pd.concat([t.iloc[:reserves_idx], t.iloc[reserves_idx+1:]])
            t.loc[:reserves_idx, 'Decision'] = 'Start'
            t.loc[reserves_idx:, 'Decision'] = 'Sit'
        else: # for pitchers
            reserves_idx = t[t['Pos']=='Reserves'].index.values[0]
            t = pd.concat([t.iloc[:reserves_idx], t.iloc[reserves_idx+1:-1]])
            t.loc[:reserves_idx, 'Decision'] = 'Start'
            t.loc[reserves_idx:t.index.max(), 'Decision'] = 'Sit'
        
        try:
            t['Team'] = t.Players.apply(lambda x: x.split()[-1])
        except:
            pass
        try:
            t['Player'] = t.Players.apply(lambda x: ' '.join(x.split()[:-3]))
        except:
            pass
        
        t = t.merge(pd.Series(id_dict).to_frame('CBSID'), left_on='Player', right_index=True, how='left')
        t = t.merge(ids[['CBSID', 'IDFANGRAPHS', 'MLBID']], on='CBSID', how='left', indicator=True)[['Player', 'Owner', 'Week', 'Team', 'Pos', 'Decision', 'CBSID', 'MLBID', 'IDFANGRAPHS']]
        df = pd.concat([df,t])
    
    df.rename(columns={'Player':'CBSNAME'},inplace=True)
    return df

In [4]:
def click_week(wk):
    details = driver.find_element_by_xpath("//div[@class ='select_form_div darkFilter']")
    res = details.find_elements_by_xpath("//li")
    for i in res:
        if i.text==wk:
            i.click()
    return

In [24]:
wk = 4
owner_id = 30
data = pd.DataFrame()
driver.get(f"https://xdl.baseball.cbssports.com/teams/{owner_id}")

#driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li[@value='/teams/roster-report/{owner_id}/1/']").click()

In [27]:
driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='{wk}']").click()

In [21]:
elem = driver.find_elements_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li")

In [39]:
html = driver.page_source
soup = bs4(html, 'html.parser')
t = pd.read_html(str(soup.find_all('table')[0:24]))[0]
t.columns = t.columns.droplevel().droplevel().droplevel()
t = t[['Pos', 'Players']]
t['Week'] = wk
t['Owner'] = owner_id
#data = pd.concat([data, scrape_cbs_owner_weekly_roster(soup, ids, owner_name, wk)])

In [43]:
t.iloc[:14]

,Pos,Players,Week,Owner
0,C,J.T. Realmuto C | PHI,4,30
1,1B,"Salvador Perez 1B,C | KC",4,30
2,2B,Ozzie Albies 2B | ATL,4,30
3,3B,"Jeimer Candelario 1B,3B | CIN",4,30
4,SS,Trea Turner SS | PHI,4,30
5,MI,Ketel Marte 2B | ARI,4,30
6,CI,Ke'Bryan Hayes 3B | PIT,4,30
7,OF,Michael Conforto OF | SF,4,30
8,OF,Lourdes Gurriel OF | ARI,4,30
9,OF,Ian Happ OF | CHC,4,30


In [26]:
t

""


In [16]:
def get_weekly_sit_or_start_by_owner(owner_name, owner_id, ids, cur_week):
    driver.get(f"https://xdl.baseball.cbssports.com/teams/{owner_id}")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")

    data = pd.DataFrame()
    for wk in range(1,cur_week):
        if wk==-1:
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='Off Season']").click()
            driver.implicitly_wait(4)
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li[@value='/teams/roster-report/{owner_id}/1/']").click()
        else:
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(.5)
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='{wk-1}']").click()
            time.sleep(2)
            elem = driver.find_elements_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li")
            driver.implicitly_wait(4)
            time.sleep(2)
            for e in elem:
                if e.text==str(wk):
                    e.click()

        time.sleep(2)
        html = driver.page_source
        soup = bs4(html, 'html.parser')
        data = pd.concat([data, scrape_cbs_owner_weekly_roster(soup, ids, owner_name, wk)])

    data.to_excel(f'data//{datetime.now().year}-{owner_name}-sit-start.xlsx', engine='openpyxl', index=False)
    return data

In [10]:
owner_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45', "Mom's Cookin":'47'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

In [8]:
ids = fp.load_id_map()

In [18]:
driver = login()
for owner_name, owner_id in owner_dict.items():
    data = get_weekly_sit_or_start_by_owner(owner_name, owner_id, ids)

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class ='select_form_div darkFilter']/div/span[text()='0']"}
  (Session info: chrome=123.0.6312.106)


In [11]:
data = get_weekly_sit_or_start_by_owner('Brewbirds', '3', ids)

In [12]:
driver.quit()

In [ ]:
name_change = {"9 Gran...":'9 Grand Kids', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 'Harvey...':'Harvey', 'Lil Tr...':'Lil Trump', 
               'Lima T...':'Lima Time', 'Roid R...':'Roiders', 'Troubl...':'Trouble', 'Ugly S...':'Ugly Spuds', 'Wiscom...':'Wu Tang', 'Young ...':'Young Guns'}

In [ ]:
team_list = ['Lima Time!', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '9 Grand Kids', 'Brewbirds', 'Trouble With The Curve', 'Lil Trump & The Ivanabees', 'Dirty Birds', 'Young Guns'] 
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [8]:
driver = login()

In [10]:
driver.get(f"https://xdl.baseball.cbssports.com/teams/3")
time.sleep(1)
driver.execute_script("window.scrollTo(0, 0);")
html = driver.page_source
soup = bs4(html, 'html.parser')

In [57]:
driver.implicitly_wait(10)
driver.get('https://xdl.baseball.cbssports.com/stats/stats-main')
time.sleep(4)
driver.find_element_by_xpath(f"//a[@href ='/SP:RP']").click()
time.sleep(1.5)
driver.find_element_by_xpath(f"//a[@href ='/all']").click()
time.sleep(1.5)
driver.execute_script("window.scrollTo(0, 150);")
driver.find_element_by_xpath(f"//div[@class ='select_div_label_container selected_arrow']").click()
time.sleep(2)
elem = driver.find_elements_by_xpath(f"//ul[@class='form_select_ul combinedFilterUl3Cols']/li")


In [68]:
for i in range(1,28):
    if len(str(i))==1:
        j = str(i)+' '
    else:
        j = str(i)
    for el in elem:
        if el.text.startswith(f'PERIOD {j}'):
            print(el.text)
        
    

PERIOD 1 (THU, MAR 30 - SUN, APR 02)
PERIOD 2 (MON, APR 03 - SUN, APR 09)
PERIOD 3 (MON, APR 10 - SUN, APR 16)
PERIOD 4 (MON, APR 17 - SUN, APR 23)
PERIOD 5 (MON, APR 24 - SUN, APR 30)
PERIOD 6 (MON, MAY 01 - SUN, MAY 07)
PERIOD 7 (MON, MAY 08 - SUN, MAY 14)
PERIOD 8 (MON, MAY 15 - SUN, MAY 21)
PERIOD 9 (MON, MAY 22 - SUN, MAY 28)
PERIOD 10 (MON, MAY 29 - SUN, JUNE 04)
PERIOD 11 (MON, JUNE 05 - SUN, JUNE 11)
PERIOD 12 (MON, JUNE 12 - SUN, JUNE 18)
PERIOD 13 (MON, JUNE 19 - SUN, JUNE 25)
PERIOD 14 (MON, JUNE 26 - SUN, JULY 02)
PERIOD 15 (MON, JULY 03 - SUN, JULY 09)
PERIOD 16 (MON, JULY 10 - SUN, JULY 16)
PERIOD 17 (MON, JULY 17 - SUN, JULY 23)
PERIOD 18 (MON, JULY 24 - SUN, JULY 30)
PERIOD 19 (MON, JULY 31 - SUN, AUG 06)
PERIOD 20 (MON, AUG 07 - SUN, AUG 13)
PERIOD 21 (MON, AUG 14 - SUN, AUG 20)
PERIOD 22 (MON, AUG 21 - SUN, AUG 27)
PERIOD 23 (MON, AUG 28 - SUN, SEP 03)
PERIOD 24 (MON, SEP 04 - SUN, SEP 10)
PERIOD 25 (MON, SEP 11 - SUN, SEP 17)
PERIOD 26 (MON, SEP 18 - SUN, SEP 24)
PER

In [14]:
h = pd.read_excel('data//2023-weekly-stats-h.xlsx')

In [15]:
pd.pivot_table()

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,MLBID,Week,Year,CBSNAME,CBSID,IDFANGRAPHS
0,CJ Abrams,22,195,Maj-NL,Washington,2,7,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.000,0.000,0.000,682928,1,2023,C.J. Abrams,3117475.0,25768
1,Jos\xc3\xa9 Abreu,36,194,Maj-AL,Houston,4,18,16,1,6,0,0,0,2,1,0,4,1,0,0,0,0,0,0.375,0.444,0.375,0.819,547989,1,2023,Jose Abreu,2106364.0,15676
2,Ronald Acuna Jr.,25,194,Maj-NL,Atlanta,3,15,13,2,3,0,0,1,1,2,0,2,0,0,0,1,1,0,0.231,0.333,0.462,0.795,660670,1,2023,Ronald Acuna,2211777.0,18401
3,Willy Adames,27,194,Maj-NL,Milwaukee,3,13,10,1,2,0,0,0,2,2,0,4,0,0,1,1,0,0,0.200,0.308,0.200,0.508,642715,1,2023,Willy Adames,2165933.0,15986
4,Jes\xc3\xbas Aguilar,33,194,Maj-AL,Oakland,3,11,10,0,3,0,0,0,0,1,1,4,0,0,0,0,0,0,0.300,0.364,0.300,0.664,542583,1,2023,Jesus Aguilar,1947830.0,11342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11067,Juan Yepez,25,13,Maj-NL,St. Louis,5,12,11,0,2,0,0,0,0,1,0,4,0,0,0,0,0,0,0.182,0.250,0.182,0.432,660766,27,2023,Juan Yepez,26793415.0,18400
11068,Masataka Yoshida,29,12,Maj-AL,Boston,5,17,17,1,5,2,0,0,1,0,0,4,0,0,0,0,0,0,0.294,0.294,0.412,0.706,807799,27,2023,Masataka Yoshida,29041593.0,31837
11069,Jacob Young,23,12,Maj-NL,Washington,5,19,18,0,5,0,0,0,3,1,0,4,0,0,0,0,5,0,0.278,0.316,0.278,0.594,696285,27,2023,NaN,NaN,NaN
11070,Jared Young,27,12,Maj-NL,Chicago,1,4,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.000,0.000,0.000,676724,27,2023,Jared Young,NaN,20404
